## Import necessary libraries

In [3]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

## Extract CSV File

In [4]:
data = pd.read_csv('resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0
1,2000-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,144000.0
2,2000-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,159000.0
3,2000-01,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,167000.0
4,2000-01,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,163000.0


#### To obtain `CombinedDataCleaned` (no outliers from both NumData and CatData)


In [5]:
NumData = pd.DataFrame(data[['floor_area_sqm', 'lease_commence_date']])


In [6]:
flat_type = pd.DataFrame(data[['flat_type']])
mapping = {'1 ROOM': 1, '2 ROOM': 2, '3 ROOM': 3, '4 ROOM': 4, '5 ROOM': 5, 'EXECUTIVE': 6, 'MULTI-GENERATION': 7}
data['flat_type_num'] = flat_type['flat_type'].map(mapping)

storey_range = pd.DataFrame(data[['storey_range']])
mapping = {'01 TO 03': 2, '04 TO 06': 5, '07 TO 09': 8, '10 TO 12': 11, '13 TO 15': 14, '16 TO 18': 17, '19 TO 21': 20, '22 TO 24': 23, '25 TO 27': 26, '28 TO 30': 29, '31 TO 33': 32, '34 TO 36': 35, '37 TO 39': 38, '40 TO 42': 41}
data['Story_range_num'] = storey_range["storey_range"].map(mapping)

flat_model = pd.DataFrame(data[['flat_model']])
mapping = {'2-room': 1, 'Adjoined flat': 2, 'Apartment': 3, 'Improved': 4, 'Improved-Maisonette': 5, 'Maisonette': 6, 'Model A': 7, 'Model A-Maisonette': 8, 'Model A2': 9, 'Multi Generation': 10, 'New Generation': 11, 'Premium Apartment': 12, 'Premium Maisonette': 13, 'Simplified': 14, 'Standard': 15, 'Terrace': 16}
data['flat_model_num'] = flat_model['flat_model'].map(mapping)

CatData = pd.DataFrame(data[['flat_type_num', 'Story_range_num','flat_model_num']])

CombinedDataCleaned = pd.concat([NumData, CatData,data['resale_price']], axis=1)

CombinedDataCleaned.info()

# Calculate the quartiles
Q1 = CombinedDataCleaned.quantile(0.25)
Q3 = CombinedDataCleaned.quantile(0.75)

# Rule to identify outliers
rule = ((CombinedDataCleaned < (Q1 - 1.5 * (Q3 - Q1))) | (CombinedDataCleaned > (Q3 + 1.5 * (Q3 - Q1))))

outliers = rule.any(axis = 1)   # axis 0 is row, 1 is column

# Which row indices correspond to outliers in the dataframe?
outlierindices = outliers.index[outliers == True]



# Remove the outliers based on the row indices obtained above
CombinedDataCleaned.drop(axis = 0,               # 0 drops row 1 drops column
                index = outlierindices, # this takes a list as input
                inplace = True)         # not overwritten by default 

# Check the clean data
CombinedDataCleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369651 entries, 0 to 369650
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   floor_area_sqm       369651 non-null  float64
 1   lease_commence_date  369651 non-null  int64  
 2   flat_type_num        369651 non-null  int64  
 3   Story_range_num      369651 non-null  int64  
 4   flat_model_num       369651 non-null  int64  
 5   resale_price         369651 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 16.9 MB
<class 'pandas.core.frame.DataFrame'>
Index: 360419 entries, 0 to 369648
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   floor_area_sqm       360419 non-null  float64
 1   lease_commence_date  360419 non-null  int64  
 2   flat_type_num        360419 non-null  int64  
 3   Story_range_num      360419 non-null  int64  
 4   flat_model_num 

In [8]:
from sklearn.model_selection import train_test_split

# Concatenate the numerical and categorical variables
X = CombinedDataCleaned.iloc[:, 0:5] # get columns 0 to 4
y = CombinedDataCleaned['resale_price']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scale the features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Define the neural network architecture
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model
mse, mae = model.evaluate(X_test, y_test)
print('Mean Squared Error:', mse)
print('Mean Absolute Error:', mae)

# Determine feature importance using permutation importance
results = permutation_importance(model, X_test, y_test, scoring='neg_mean_squared_error', n_repeats=10, random_state=0)
importance = results.importances_mean

# Print the feature importance scores
for i, v in enumerate(importance):
    print('Feature %d, Score: %.5f' % (i, v))

Epoch 1/100
9011/9011 [==============================] - 10s 1ms/step - loss: 21015764992.0000 - mae: 106118.7500
Epoch 2/100
9011/9011 [==============================] - 10s 1ms/step - loss: 4912357376.0000 - mae: 55525.8711
Epoch 3/100
9011/9011 [==============================] - 9s 1ms/step - loss: 4823678976.0000 - mae: 55099.3984
Epoch 4/100
9011/9011 [==============================] - 9s 1ms/step - loss: 4778604032.0000 - mae: 54847.5273
Epoch 5/100
9011/9011 [==============================] - 9s 1ms/step - loss: 4746759680.0000 - mae: 54658.3945
Epoch 6/100
9011/9011 [==============================] - 9s 1ms/step - loss: 4726799872.0000 - mae: 54547.5273
Epoch 7/100
9011/9011 [==============================] - 10s 1ms/step - loss: 4712401408.0000 - mae: 54462.1250
Epoch 8/100
9011/9011 [==============================] - 9s 1ms/step - loss: 4702331904.0000 - mae: 54405.8125
Epoch 9/100
9011/9011 [==============================] - 9s 1ms/step - loss: 4694575104.0000 - mae: 54361.2